In [10]:
import pandas as pd
import numpy as np
from numpy import array as a
from random import random
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle


# input dataset
filename1 = "BIG_DWH_Training.csv"
df1 = pd.read_csv( filename1, header=None )
df1.rename(columns={0: 'Index', 1: 'Height', 2: 'Weight', 3:'Gender'}, inplace=True)
df1 = df1.drop(columns = "Index")

data1=df1.replace({'Gender': -1}, 0)
data1 = shuffle(data1)

#normalize data
normalize = preprocessing.StandardScaler().fit(data1.iloc[:, 1:3].values)
new_df1 = normalize.transform(data1.iloc[:, 1:3].values)
data1['Height'] = new_df1[:, 0]
data1['Weight'] = new_df1[:, 1]

#divide data into training and validation sets
train, validation = train_test_split(data1, test_size=0.05)

featuresTrain = train[['Height','Weight']].values.tolist()
featuresTrain = a(featuresTrain)
genderTrain = train[['Gender']].values.tolist()
genderTrain = a(genderTrain)

featuresVal = validation[['Height','Weight']].values.tolist()
featuresVal = a(featuresVal)
genderVal = validation[['Gender']].values.tolist()
genderVal = a(genderVal)

#test data set
filename2 = "DWH_Test.csv"
df2 = pd.read_csv( filename2, header=None )
df2.rename(columns={0: 'Index', 1: 'Height', 2: 'Weight', 3:'Gender'}, inplace=True)
df2 = df2.drop(columns = "Index")

data2=df2.replace({'Gender': -1}, 0)

#normalize data
normalize = preprocessing.StandardScaler().fit(data2.iloc[:, 1:3].values)
new_df2 = normalize.transform(data2.iloc[:, 1:3].values)
data2['Height'] = new_df2[:, 0]
data2['Weight'] = new_df2[:, 1]

featuresTest = data2[['Height','Weight']].values.tolist()
featuresTest = a(featuresTest)
genderTest = data2[['Gender']].values.tolist()
genderTest = a(genderTest)

In [12]:
class NeuralNetwork():
    
    #fn for initial weight creation of NN
    def create_network(neuron_in, neuron_out):    
        layer = np.random.randn(neuron_out,neuron_in) * np.sqrt(2/neuron_out)  
        return layer
    
    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(-x))
    
    # Forward propagate: takes weights and features and returns activation of each neuron in every layer of NN
    def forward_propogation(features, ann1, ann2, ann3):
        activation = [] #list
        activation1 = []
        activation2 = []
        
        for row in features:
            act1 = np.dot(ann1, row)
            act1 = NeuralNetwork.sigmoid(act1) 
            activation1.append(act1)
            
            act2 = np.dot(ann2, act1)
            act2 = NeuralNetwork.sigmoid(act2)
            activation2.append(act2)
            
            act3 = np.dot(ann3, act2)
            act3 = NeuralNetwork.sigmoid(act3)
            activation.append(act3)
            
        activation1 = a(activation1)
        activation2 = a(activation2)
        activation = a(activation)

        return activation1, activation2, activation
    
    def sigmoid_derivative(x):
        return x * (1 - x)
    
    # Backpropagate : Computes gradient of each layer and error of every iteration
    def backward_propagation(ann1, ann2, ann3, actual, a1, a2, a3):        
        error = []
        dError = []
        sigDer3 = []
        delta3 = []
        gradient3=[]
        delta2 = []
        gradient2=[]
        sigDer2=[]
        delta1 = []
        gradient1=[]
        sigDer1=[]
        
        for i in range(len(actual)):
            
            error.append(-(np.multiply(actual[i],np.log(a3[i])) + np.multiply(1-actual[i],np.log(1-a3[i]))))
            dError.append(-((actual[i]/a3[i]) - ((1-actual[i])/(1-a3[i]))))
            sigDer3.append(NeuralNetwork.sigmoid_derivative(a3[i]))
            sigDer2.append(NeuralNetwork.sigmoid_derivative(a2[i]))
            sigDer1.append(NeuralNetwork.sigmoid_derivative(a1[i]))
            delta3.append(dError[i] * sigDer3[i])
            
        error= np.mean(error)

        sigDer2 = a(sigDer2)  
        sigDer1 = a(sigDer1)
        
        delta3 = a(delta3)      
        gradient3 = np.multiply(delta3,a3)
        gradient3 = np.divide(np.mean(gradient3),50)
        
        delta2 = np.dot(delta3,ann3)
        delta2 = np.multiply(delta2,sigDer2)
        gradient2 = np.multiply(delta2,a2)
        gradient2 = np.divide(np.mean(gradient2),50)
   
        delta1 = np.dot(delta2,ann2)
        delta1 = np.multiply(delta1,sigDer1)
        gradient1 = np.multiply(delta1,a1)
        gradient1 = np.divide(np.mean(gradient1),50)
        
        return gradient1, gradient2, gradient3, error   
    
    #updates weights based on Ψ, weights, gradients, and learning rate
    def update_weights(syi, ann1, ann2, ann3, lr, g1, g2, g3): 
        
        dell1= syi* dell1 - g1
        term1 = lr*dell1
        
        dell2= syi* dell2 - g2
        term2 = lr*dell2
        
        dell3= syi* dell3 - g3
        term3 = lr*dell3
        
        
        weights1 = np.subtract(ann1, term1)
        weights2 = np.subtract(ann2, term2)
        weights3 = np.subtract(ann3, term3)
        
        return weights1, weights2, weights3

    
    
if __name__ == "__main__":
    
    ann1 = NeuralNetwork.create_network(2,5)
    ann2 = NeuralNetwork.create_network(5,5) 
    ann3 = NeuralNetwork.create_network(5,1)
    
    batchSizeTrain = 0
#     batchSizeVal = 0
    valErrorList = np.empty(0)
#     trainAccuracy = np.empty(0)
    valAccuracy = np.empty(0)
    out = True
    x=[5,7,11]
    
    for value in x:
        print("λ : ", value)
#         syi= np.random.random(1)[0]
        syi=0.9
        print("Ψ :", syi)
    
        for i in range(1,1550):
            if i%250 != 0:      #training
#                 print("Inside Training batch", i)  
                featuresTrainSplit = featuresTrain[batchSizeTrain:batchSizeTrain+50]
                genderTrainSplit = genderTrain[batchSizeTrain:batchSizeTrain+50]
                activation1, activation2, activation3 = NeuralNetwork.forward_propogation(featuresTrainSplit, ann1, ann2, ann3)
                avg_gradient1, avg_gradient2, avg_gradient3, avg_error = NeuralNetwork.backward_propagation(ann1, ann2, ann3, genderTrainSplit, activation1, activation2, activation3)
#                 print("Error :", avg_error)

                resultTrain = 0
                predictTrain= np.zeros(50)

                for index in range(len(activation3)): 
                    if (activation3[index] < 0.5) :      #female
                        predictTrain[index] = 0
                        if (genderTrainSplit[index] == predictTrain[index]): 
                            resultTrain += 1                      
                    else:               #classify as male
                        predictTrain[index] = 1    
                        if (genderTrainSplit[index] == predictTrain[index]): 
                            resultTrain += 1
                t_acc = (resultTrain/50) * 100
#                 print("Accuracy : ", t_acc ,"%")
#                 print("")
                trainAccuracy = np.append(trainAccuracy, t_acc)
                
                ann1, ann2, ann3 = NeuralNetwork.update_weights(syi, ann1, ann2, ann3, value/i, avg_gradient1, avg_gradient2, avg_gradient3)
                batchSizeTrain += 50

            else:   #validation
#                 print("")
#                 print("Inside Validation batch", i)
                featuresValSplit = featuresVal[batchSizeVal:batchSizeVal+50]
                genderValSplit = genderVal[batchSizeVal:batchSizeVal+50]
                activation1, activation2, activation3 = NeuralNetwork.forward_propogation(featuresValSplit, ann1, ann2, ann3)
                avg_gradient1, avg_gradient2, avg_gradient3, avg_error = NeuralNetwork.backward_propagation(ann1, ann2, ann3, genderValSplit, activation1, activation2, activation3)
#                 print("Error: ", avg_error)

                if len(valErrorList) % 5 == 0:
                    print("Last 5 validation error: ", valErrorList[-5:])
#                     print("Error: ", avg_error)
                    out = np.greater(valErrorList[-5:], avg_error)
                    if False in out:
                        break

                valErrorList = np.append(valErrorList, avg_error)

                resultVal = 0
                predictVal = np.zeros(50)

                for index in range(len(activation3)):
                    if (activation3[index] < 0.5) :      #female
                        predictVal[index] = 0
                        if (genderValSplit[index] == predictVal[index]): 
                            resultVal += 1                      
                    else:               #classify as male
                        predictVal[index] = 1    
                        if (genderValSplit[index] == predictVal[index]): 
                            resultVal += 1
                v_acc = (resultVal/50) * 100 
#                 print("Accuracy : ", v_acc ,"%")
#                 print("")

                valAccuracy = np.append(valAccuracy, v_acc)

                ann1, ann2, ann3 = NeuralNetwork.update_weights(syi, ann1, ann2, ann3, value/i, avg_gradient1, avg_gradient2, avg_gradient3)
                batchSizeVal += 50 

        print("\nTraining accuracies:", trainAccuracy)
        print("Validation accuracies:", valAccuracy)
        print("End of training for λ =", value,". Weights found to be as below:- ")
        print("\nLayer 1:\n ", ann1, " \n Layer 2:\n ",ann2,"", "\n Layer 3: \n", ann3)
        
        #testing set
        activation1, activation2, activation3 = NeuralNetwork.forward_propogation(featuresTest, ann1, ann2, ann3)
        resultTest = 0
        predictTest= np.zeros(45)

        for index in range(len(genderTest)): 
            if (activation3[index] < 0.5): #female
                predictTest[index] = 0
                if (genderTest[index] == predictTest[index]): 
                    resultTest += 1                      
            else:               #classify as male
                predictTest[index] = 1   
                if (genderTest[index] == predictTest[index]): 
                    resultTest += 1
        t_acc = (resultTest/45) * 100  
       
        print("Accuracy : ", t_acc ,"%")
        print("")
    value += 1 #next value of λ
    

λ :  5
Ψ : 0.9
Last 5 validation error:  []
Last 5 validation error:  [0.54583879 0.76475983 0.70158324 0.72972112 0.66352204]

Training accuracies: [46. 66. 38. ... 56. 48. 50.]
Validation accuracies: [70. 38. 46. 42. 52.]
End of training for λ = 5 . Weights found to be as below:- 

Layer 1:
  [[ 0.37516271  0.15747002]
 [-0.20290279 -0.3314144 ]
 [-0.17221443 -0.19179876]
 [-1.00707421 -0.22825695]
 [ 0.49845841 -0.19935595]]  
 Layer 2:
  [[-0.64395945  0.23942659  0.26021896  0.50301353  1.34051966]
 [ 0.26996663 -0.24704966 -0.5202977  -0.03175176 -0.2309165 ]
 [-0.82112341  0.02435452 -0.30686432  1.49454632  0.05933619]
 [-0.25280571 -0.48894054  0.03443715  0.04199326 -1.15255846]
 [ 0.18938483  0.1905991   0.80152569 -0.4563081  -0.25455455]]  
 Layer 3: 
 [[-2.17816289  2.12634903  0.60326951  1.79366829  0.89544965]]
Accuracy :  55.55555555555556 %

λ :  7
Ψ : 0.9
Last 5 validation error:  [0.54583879 0.76475983 0.70158324 0.72972112 0.66352204]

Training accuracies: [46. 66